<a href="https://colab.research.google.com/github/riyasachdeva04/coffee-chatbot/blob/main/riiibott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence =[stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words), dtype = np.float32)

  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  return bag

  

# a = "How long does shipping take?"
# print(a)
# a = tokenize(a)
# print(a)

# words = ["wished", "wishing", "wishes"]
# stemmed_words =[stem(w) for w in words]
# print(stemmed_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import files
uploaded = files.upload()

Saving riiibott.json to riiibott (4).json


In [3]:
import pandas as pd
import io
intents = pd.read_json(io.BytesIO(uploaded['riiibott.json']))

In [4]:
# training
import json
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

with open('riiibott.json', 'r') as f:
  intents = json.load(f)

print(intents)
all_words = []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))

ignore_words = ['?', ',', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 

batch_size = 8


class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'name', 'patterns': ['Tell your name', 'What is your name', 'Who are you', 'Tell me about yourself'], 'responses': ['My name is riiibott and I am sexy.', 'I am riiibott, I am very cool.', 'My name is riiibott and my twt is @riiiwtff']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'pa

In [5]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    return out


In [6]:
import torch

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

59 8
Epoch [100/1000], Loss: 1.6619
Epoch [200/1000], Loss: 0.6691
Epoch [300/1000], Loss: 0.0874
Epoch [400/1000], Loss: 0.0635
Epoch [500/1000], Loss: 0.0314
Epoch [600/1000], Loss: 0.0030
Epoch [700/1000], Loss: 0.0032
Epoch [800/1000], Loss: 0.0036
Epoch [900/1000], Loss: 0.0021
Epoch [1000/1000], Loss: 0.0006
final loss: 0.0006


In [7]:
data = {
    "model_state" : model.state_dict(),
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "output_size" : output_size,
    "all_words" : all_words,
    "tags" : tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f"Training complete, file saved to {FILE}")

Training complete, file saved to data.pth


In [8]:
import random
import json
import torch

data = torch.load(FILE)
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=59, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
)

In [10]:
bot_name = "riiibott"
print("Let's chat! type 'quit to exit")

while True:
  sentence = input("You: ")
  if sentence == "quit":
    break
  
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{bot_name}: I do not understand...")

Let's chat! type 'quit to exit
You: who are you?
riiibott: My name is riiibott and I am sexy.
You: what do you sell?
riiibott: We have coffee and tea
You: how long do you take to deliver?
riiibott: Hi there, what can I do for you?
You: how long does delivery take?
riiibott: Shipping takes 2-4 days
You: tell me a joke
riiibott: What did the buffalo say when his son left for college? Bison.
You: bye son
riiibott: Bye! Come back again soon.
You: quit
